## STEP 1 — Load ast_index.jsonl di LangChain

In [21]:
from langchain_community.document_loaders import JSONLoader

loader = JSONLoader(
    file_path="../data/ast_data/ast_index.jsonl",
    jq_schema=".",
    content_key="content",
    metadata_func=lambda record, metadata: record["metadata"],
    json_lines=True
)

docs = loader.load()
docs
# print(docs[0].page_content)
# print(docs[0].metadata)
# print(docs[0])

[Document(metadata={'Name': 'IOrderService', 'Namespace': 'Application', 'Layer': 'Application', 'IsAggregateRoot': False, 'Type': 'InterfaceDeclaration'}, page_content='Class: IOrderService\r\nNamespace: Application\r\nLayer: Application\r\nAggregateRoot: False\r\n\r\nProperties:\r\n\r\n\r\nMethods:\r\nTask CreateOrderApprovalAsync()'),
 Document(metadata={'Name': 'OrderService', 'Namespace': 'Application', 'Layer': 'Application', 'IsAggregateRoot': False, 'Type': 'ClassDeclaration'}, page_content='Class: OrderService\r\nNamespace: Application\r\nLayer: Application\r\nAggregateRoot: False\r\n\r\nProperties:\r\n\r\n\r\nMethods:\r\n'),
 Document(metadata={'Name': 'WeatherForecast', 'Namespace': 'OrderService', 'Layer': 'Presentation', 'IsAggregateRoot': False, 'Type': 'ClassDeclaration'}, page_content='Class: WeatherForecast\r\nNamespace: OrderService\r\nLayer: Presentation\r\nAggregateRoot: False\r\n\r\nProperties:\r\nDateOnly Date\nint TemperatureC\nint TemperatureF\nstring? Summary\r

## STEP 3 — Embedding ke Chroma (Ollama)

In [22]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    persist_directory="./chroma_ast"
)

vectorstore.persist()


C:\Users\wibow\AppData\Local\Temp\ipykernel_22352\1185385336.py:10: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


## STEP 4 — Query dengan FILTER AST

In [38]:
retriever = vectorstore.as_retriever(
    search_kwargs={
        "k": 5,
        "filter": {
            "Layer": "Domain",
            "IsAggregateRoot": True
        }
    }
)

context_docs = retriever.invoke("Order AggregateRoot")
print(context_docs)

ValueError: Expected where to have exactly one operator, got {'Layer': 'Domain', 'IsAggregateRoot': True} in query.

In [39]:
retriever = vectorstore.as_retriever(
    search_kwargs={
        "k": 5,
        "filter": {
            "Layer": "Domain",
            "IsAggregateRoot": True
        }
    }
)
print(retriever)

tags=['Chroma', 'OllamaEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000020D988B9130> search_kwargs={'k': 5, 'filter': {'Layer': 'Domain', 'IsAggregateRoot': True}}


## 🤖 STEP 7 — RAG Chain (Ollama Chat)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama


llm = ChatOllama(
    model="llama3.1",
    temperature=0
)


# prompt = ChatPromptTemplate.from_template("""
# Kamu adalah senior C# architect.
# Ikuti Clean Architecture:
# - Domain tidak bergantung ke layer lain
# - Application berisi Use Case
# - Infrastructure hanya implementasi

# 1. Gunakan pola yang SUDAH ADA di repository.
# 2. Gunakan struktur AST berikut untuk membuat Use Case

# Context:
# {context}

# Question:
# {question}
# """)
prompt =ChatPromptTemplate.from_template("""
Kamu adalah senior C# architect.

Context AST (Domain & Application):
{context}

Buatkan:
1. Command (record)
2. Handler (implements IRequestHandler)
3. Interface Repository (Domain)

Rules:
- Clean Architecture
- Domain tidak bergantung ke Application
- Gunakan pola existing di repo
- Jangan membuat logic di Controller

Question:
{question}
""")

retriever = vectorstore.as_retriever(
    search_kwargs={
        "k": 6,
        "filter": {
            "Layer": {"$in": ["Domain", "Application"]}
        }
    }
)

rag_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
)


In [41]:
query_orderApproval = """
Buatkan Use Case untuk Approve Order
berdasarkan pola yang sudah ada di Application layer
dan entity di Domain
"""
result = rag_chain.invoke(
   query_orderApproval
)

print(result.content)

Berikut adalah implementasi yang sesuai dengan aturan-aturan yang telah ditentukan:

**1. Command (record)**

Membuat command `CreateOrderCommand` sebagai record:
```csharp
public class CreateOrderCommand : ICommand
{
    public Guid Id { get; }
    public string CustomerName { get; }
    public IReadOnlyCollection<OrderItem> Items { get; }

    public CreateOrderCommand(Guid id, string customerName, IReadOnlyCollection<OrderItem> items)
    {
        Id = id;
        CustomerName = customerName;
        Items = items;
    }
}
```
**2. Handler (implements IRequestHandler)**

Membuat handler `CreateOrderHandler` yang mengimplementasikan `IRequestHandler<CreateOrderCommand>`:
```csharp
public class CreateOrderHandler : IRequestHandler<CreateOrderCommand>
{
    private readonly IOrderRepository _repository;

    public CreateOrderHandler(IOrderRepository repository)
    {
        _repository = repository;
    }

    public async Task HandleAsync(CreateOrderCommand command)
    {
        v